## Internship - MICS - CentraleSupelec - Felipe Moret & Ioane Muni Toke

In [1]:
import pandas as pd
import datetime as dt
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

**Investigating tick by tick data - Reconstruction of Order Flow using Aggregated Data**

**Reading Data**

In [2]:
quote_data_path = "/Users/felipemoret/Desktop/first_task_supelec/FR/book/BNPP.PA/2016-01-04-BNPP.PA-book.csv.gz"
trade_data_path = "/Users/felipemoret/Desktop/first_task_supelec/FR/trade/BNPP.PA/2016-01-04-BNPP.PA-trade.csv.gz"


##Reading order book (quotes) data
quote_data = pd.read_csv(quote_data_path, compression ='gzip', error_bad_lines = False)
trade_data = pd.read_csv(trade_data_path, compression ='gzip', error_bad_lines = False)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Correcting columns' index for quotes data**

In [3]:
quote_data.head()

,xltime,Unnamed: 1,bid-price-1,bid-volume-1,ask-price-1,ask-volume-1,bid-price-2,bid-volume-2,ask-price-2,ask-volume-2,bid-price-3,bid-volume-3,ask-price-3,ask-volume-3,bid-price-4,bid-volume-4,ask-price-4,ask-volume-4,bid-price-5,bid-volume-5,ask-price-5,ask-volume-5,bid-price-6,bid-volume-6,ask-price-6,ask-volume-6,bid-price-7,bid-volume-7,ask-price-7,ask-volume-7,bid-price-8,bid-volume-8,ask-price-8,ask-volume-8,bid-price-9,bid-volume-9,ask-price-9,ask-volume-9,bid-price-10,bid-volume-10,ask-price-10,ask-volume-10
0,42373.218982,43.5,70,69.0,166,43.5,170,69.0,166,38.8,1500,69.0,166,37.8,1500,69.0,166,37.8,3000,69.0,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),NaN
1,42373.218982,43.5,70,69.0,166,43.5,170,69.0,166,38.8,1500,69.0,166,37.8,1500,69.0,166,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),NaN
2,42373.218982,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),NaN
3,42373.218982,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),NaN
4,42373.218982,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),NaN


In [4]:
quote_data_columns = quote_data.columns
quote_data_first_column = quote_data_columns[0] 
quote_data_columns = quote_data.columns[2:]
quote_data_columns = quote_data_columns.insert(0, quote_data_first_column)

In [5]:
quote_data = quote_data.iloc[: , :-1]

In [6]:
quote_data.columns = quote_data_columns

In [7]:
quote_data.head()

,xltime,bid-price-1,bid-volume-1,ask-price-1,ask-volume-1,bid-price-2,bid-volume-2,ask-price-2,ask-volume-2,bid-price-3,bid-volume-3,ask-price-3,ask-volume-3,bid-price-4,bid-volume-4,ask-price-4,ask-volume-4,bid-price-5,bid-volume-5,ask-price-5,ask-volume-5,bid-price-6,bid-volume-6,ask-price-6,ask-volume-6,bid-price-7,bid-volume-7,ask-price-7,ask-volume-7,bid-price-8,bid-volume-8,ask-price-8,ask-volume-8,bid-price-9,bid-volume-9,ask-price-9,ask-volume-9,bid-price-10,bid-volume-10,ask-price-10,ask-volume-10
0,42373.218982,43.5,70,69.0,166,43.5,170,69.0,166,38.8,1500,69.0,166,37.8,1500,69.0,166,37.8,3000,69.0,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
1,42373.218982,43.5,70,69.0,166,43.5,170,69.0,166,38.8,1500,69.0,166,37.8,1500,69.0,166,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
2,42373.218982,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
3,42373.218982,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
4,42373.218982,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()


**Converting excel datetime for Paris datetime**

In [8]:
quote_data['fulldate'] = pd.TimedeltaIndex(quote_data['xltime'], unit = 'd') + dt.datetime(1899,12,30)
quote_data = quote_data.set_index('fulldate').tz_localize('UTC').tz_convert('Europe/Paris').tz_localize(None)
#quote_data = quote_data.reset_index()

In [9]:
quote_data = quote_data.drop("xltime", axis = 1)

In [10]:
quote_data.head()

,bid-price-1,bid-volume-1,ask-price-1,ask-volume-1,bid-price-2,bid-volume-2,ask-price-2,ask-volume-2,bid-price-3,bid-volume-3,ask-price-3,ask-volume-3,bid-price-4,bid-volume-4,ask-price-4,ask-volume-4,bid-price-5,bid-volume-5,ask-price-5,ask-volume-5,bid-price-6,bid-volume-6,ask-price-6,ask-volume-6,bid-price-7,bid-volume-7,ask-price-7,ask-volume-7,bid-price-8,bid-volume-8,ask-price-8,ask-volume-8,bid-price-9,bid-volume-9,ask-price-9,ask-volume-9,bid-price-10,bid-volume-10,ask-price-10,ask-volume-10
fulldate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-04 06:15:20.042035200,43.5,70,69.0,166,43.5,170,69.0,166,38.8,1500,69.0,166,37.8,1500,69.0,166,37.8,3000,69.0,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
2016-01-04 06:15:20.042035200,43.5,70,69.0,166,43.5,170,69.0,166,38.8,1500,69.0,166,37.8,1500,69.0,166,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
2016-01-04 06:15:20.042035200,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
2016-01-04 06:15:20.042035200,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()
2016-01-04 06:15:20.042035200,43.5,70,61.5,22,43.5,170,69.0,166,38.8,3000,68.5,50,37.8,3000,68.5,50,37.8,3000,69.0,166,(),(),69,166,(),(),(),(),(),(),(),(),(),(),(),(),(),(),(),()


**Correcting columns' index for trades data**

In [11]:
trade_data.columns = ['ts', 'price', 'volume', 'flag', 'rawflag']

In [12]:
trade_data.head(5)

,ts,price,volume,flag,rawflag
0,42373.100934,52.59,0.0,settlementprice,[BNPP]Settlement Price
1,42373.231234,52.23,0.0,settlementprice,[BNPP]Settlement Price
2,42373.260417,52.23,0.0,auctionphase,[BNPP]Auction Phase
3,42373.260418,52.23,0.0,theoricalprice|auctionphase,[PRC_QL_CD]IND[BNPP]Auction Phase
4,42373.260735,52.20,50.0,auctionphase,[BNPP]Auction Phase


**FIltering trade data for only "uncategorized" flag**

In [13]:
trade_data = trade_data[trade_data["flag"] == "uncategorized"]

In [14]:
trade_data.shape

(20200, 5)

**Converting excel datetime for Paris datetime**

In [15]:
trade_data['fulldate'] = pd.TimedeltaIndex(trade_data['ts'], unit = 'd') + dt.datetime(1899,12,30)
trade_data = trade_data.set_index('fulldate').tz_localize('UTC').tz_convert('Europe/Paris').tz_localize(None)
#trade_data = trade_data.reset_index()

In [16]:
trade_data = trade_data.drop("ts", axis = 1)

In [17]:
trade_data.head(5)

,price,volume,flag,rawflag
fulldate,,,,
2016-01-04 09:00:29.483971200,51.75,47.0,uncategorized,[ACT_FLAG1] [PRC_QL_CD]
2016-01-04 09:00:29.483971200,51.75,26.0,uncategorized,[ACT_FLAG1]
2016-01-04 09:00:29.483971200,51.75,1.0,uncategorized,[ACT_FLAG1]
2016-01-04 09:00:29.483971200,51.75,47.0,uncategorized,[ACT_FLAG1]
2016-01-04 09:00:29.483971200,51.75,50.0,uncategorized,[ACT_FLAG1]


**Analyzing number of repeated records**

In [18]:
print(trade_data.duplicated().sum())

8821


## Parsing the Quotes Data

**Analyzing the number of missing data**

In [19]:
quote_data.isna().sum()

bid-price-1      0
bid-volume-1     0
ask-price-1      0
ask-volume-1     0
bid-price-2      0
bid-volume-2     0
ask-price-2      0
ask-volume-2     0
bid-price-3      0
bid-volume-3     0
ask-price-3      0
ask-volume-3     0
bid-price-4      0
bid-volume-4     0
ask-price-4      0
ask-volume-4     0
bid-price-5      0
bid-volume-5     0
ask-price-5      0
ask-volume-5     0
bid-price-6      0
bid-volume-6     0
ask-price-6      0
ask-volume-6     0
bid-price-7      0
bid-volume-7     0
ask-price-7      0
ask-volume-7     0
bid-price-8      0
bid-volume-8     0
ask-price-8      0
ask-volume-8     0
bid-price-9      0
bid-volume-9     0
ask-price-9      0
ask-volume-9     0
bid-price-10     0
bid-volume-10    0
ask-price-10     0
ask-volume-10    0
dtype: int64

**Shape of the quotes data**

In [20]:
quote_data.shape

(504824, 40)

**Analyzing number of repeated records**

In [21]:
print(quote_data.duplicated().sum())

11695


**Removing duplicated rows**

In [22]:
quote_data = quote_data[~quote_data.duplicated()]

In [23]:
quote_data.shape

(493129, 40)

**Fixing a time period within the day for working...**

In [24]:
##initial_period = trade_data.index[0]
initial_period = pd.Timestamp('2016-01-04 09:10:00.00')

In [25]:
initial_period

Timestamp('2016-01-04 09:10:00')

In [26]:
##final_period = trade_data.index[-1]
final_period = pd.Timestamp('2016-01-04 09:11:00.00')

In [27]:
final_period

Timestamp('2016-01-04 09:11:00')

**Getting quote_data and trade_data in the specified [initial_period, final_period]**

In [28]:
mask_quote = (quote_data.index > initial_period) & (quote_data.index <= final_period)
reduced_quote_data = quote_data[mask_quote] 

In [29]:
reduced_quote_data.shape

(1165, 40)

In [30]:
mask_quote = (trade_data.index > initial_period) & (trade_data.index <= final_period)
reduced_trade_data = trade_data[mask_quote] 

In [31]:
reduced_trade_data.shape

(46, 4)

**Function to find LIMIT ORDER OR (A PRIORI) CANCELLATIONS in the Limit Order Book**

In [32]:
def return_changes_between_timestamps(reduced_quote_data):
        
    reduced_quote_data = reduced_quote_data.apply(pd.to_numeric)
    parsed_order_book = reduced_quote_data.diff()
    
    return parsed_order_book    

In [33]:
parsed_order_book = return_changes_between_timestamps(reduced_quote_data)

In [34]:
parsed_order_book = parsed_order_book.iloc[1:]

In [35]:
parsed_order_book.shape

(1164, 40)

In [36]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

In [37]:
def classify_changes_parsed_order_book_before_trading_match(reduced_quote_data, parsed_order_book):
    
    prices_bid_indices = [0,4,8,12,16,20,24,28,32,36]
    prices_ask_indices = [2,6,10,14,18,22,26,30,34,38]
    volume_bid_indices = [1,5,9,13,17,21,25,29,33,37]
    volume_ask_indices = [3,7,11,15,19,23,27,31,35,39]
        
    columns_names = ["Timestamp", "Level", "Side", "Price", "Quantity", "Flow"]
    before_trade_match_order_flow = pd.DataFrame( columns = columns_names)
    
    number_of_records = parsed_order_book.shape[0]
    
    ## Analyzing timestamps where only one change took place (one volume column)
    for i in range(0, number_of_records):
        change_indexes = list(parsed_order_book.iloc[i].to_numpy().nonzero()[0])
        if(intersection(change_indexes, (prices_bid_indices + prices_ask_indices)) == []):
            for volume_change_column_index in change_indexes:
                quantity = parsed_order_book.iloc[i].values[volume_change_column_index]
                if volume_change_column_index in volume_bid_indices:
                    level = volume_bid_indices.index(volume_change_column_index) + 1
                    side = "BID"
                else:
                    level = volume_ask_indices.index(volume_change_column_index) + 1
                    side = "ASK"
                if quantity > 0:
                    flow = "LIMIT ORDER"
                else:
                    flow = "CANCELLATION"
                timestamp = parsed_order_book.iloc[i].name
                price = reduced_quote_data.iloc[i+1].values[volume_change_column_index - 1]
                to_append = [timestamp, level, side, price, quantity, flow]
                to_append_series = pd.Series(to_append, index = before_trade_match_order_flow.columns)
                before_trade_match_order_flow  = before_trade_match_order_flow.append(to_append_series, ignore_index = True)
        else:
            for price_change_column_index in intersection(change_indexes, prices_bid_indices + prices_ask_indices):
                if(price_change_column_index in prices_bid_indices):
                    level = prices_bid_indices.index(price_change_column_index) + 1
                    side = "BID"  
                else:
                    level = prices_ask_indices.index(price_change_column_index) + 1
                    side = "ASK"
                                     
                price = reduced_quote_data.iloc[i+1].values[price_change_column_index]
   
                if(level == 1):
                    price_in_line_above_posterior_level = reduced_quote_data.iloc[i].values[price_change_column_index + 4]
                    if(float(price) ==  float(price_in_line_above_posterior_level)):
                        quantity = float(reduced_quote_data.iloc[i+1].values[price_change_column_index + 1]) - float(reduced_quote_data.iloc[i].values[price_change_column_index + 1 + 4])
                        price_in_line_above_same_level = reduced_quote_data.iloc[i].values[price_change_column_index]
                        quantity_in_live_above_same_level = float(reduced_quote_data.iloc[i].values[price_change_column_index + 1])
                        to_append = [timestamp, level, side, price_in_line_above_same_level, quantity_in_live_above_same_level, "CANCELLATION"]
                        to_append_series = pd.Series(to_append, index = before_trade_match_order_flow.columns)
                        before_trade_match_order_flow  = before_trade_match_order_flow.append(to_append_series, ignore_index = True)
                    else:
                        quantity = reduced_quote_data.iloc[i+1].values[price_change_column_index + 1]
                    quantity = float(quantity)
                    
                elif(level == 10):
                    price_in_line_above_previous_level = reduced_quote_data.iloc[i].values[price_change_column_index - 4]
                    if(float(price) ==  float(price_in_line_above_previous_level)):
                        quantity = float(reduced_quote_data.iloc[i+1].values[price_change_column_index + 1]) - float(reduced_quote_data.iloc[i].values[price_change_column_index + 1 - 4])
                    else:
                        if(i >= 1):
                            price_in_two_lines_above_same_level =  reduced_quote_data.iloc[i-1].values[price_change_column_index]
                            if(float(price) ==  float(price_in_two_lines_above_same_level)):
                                quantity = float(reduced_quote_data.iloc[i+1].values[price_change_column_index + 1]) - float(reduced_quote_data.iloc[i-1].values[price_change_column_index + 1])
                        else:
                            quantity = "ERROR"
                            flow = "ERROR"              
                else:
                    price_in_line_above_posterior_level = reduced_quote_data.iloc[i].values[price_change_column_index + 4]
                    price_in_line_above_previous_level = reduced_quote_data.iloc[i].values[price_change_column_index - 4]
                    price_in_line_above_same_level = reduced_quote_data.iloc[i].values[price_change_column_index]
                    price_in_same_line_posterior_level = reduced_quote_data.iloc[i+1].values[price_change_column_index + 4]
                    if(float(price) ==  float(price_in_line_above_previous_level)):
                        quantity = float(reduced_quote_data.iloc[i+1].values[price_change_column_index + 1]) - float(reduced_quote_data.iloc[i].values[price_change_column_index + 1 - 4])
                    elif(float(price) ==  float(price_in_line_above_posterior_level)):
                        quantity = float(reduced_quote_data.iloc[i+1].values[price_change_column_index + 1]) - float(reduced_quote_data.iloc[i].values[price_change_column_index + 1 + 4])
                    
                    elif(price_in_line_above_same_level == price_in_same_line_posterior_level):
                        quantity = float(reduced_quote_data.iloc[i+1].values[price_change_column_index + 1])
                    else:
                        quantity = "ERROR"
                        flow = "ERROR"
                    
                if(quantity != "ERROR" and quantity > 0):
                    flow = "LIMIT ORDER"
                if(quantity != "ERROR" and  quantity < 0):
                    flow = "CANCELLATION"
                timestamp = parsed_order_book.iloc[i].name
                to_append = [timestamp, level, side, price, quantity, flow]
                to_append_series = pd.Series(to_append, index = before_trade_match_order_flow.columns)
                if(quantity != 0):
                    before_trade_match_order_flow  = before_trade_match_order_flow.append(to_append_series, ignore_index = True)
                
            remaining_change_indexes = list(set(change_indexes) - set(intersection(change_indexes, prices_bid_indices + prices_ask_indices)))
            number_of_indexes_volumes_already_checked = len(intersection(change_indexes, prices_bid_indices + prices_ask_indices))
            volumes_already_checked_indexes = intersection(change_indexes, prices_bid_indices + prices_ask_indices) + np.ones(number_of_indexes_volumes_already_checked)
            remaining_change_indexes = list(set(remaining_change_indexes) - set(volumes_already_checked_indexes))
            

            for volume_change_column_index in remaining_change_indexes:
                quantity = parsed_order_book.iloc[i].values[volume_change_column_index]
                if volume_change_column_index in volume_bid_indices:
                    level = volume_bid_indices.index(volume_change_column_index) + 1
                    side = "BID"
                else:
                    level = volume_ask_indices.index(volume_change_column_index) + 1
                    side = "ASK"
                if quantity > 0:
                    flow = "LIMIT ORDER"
                else:
                    flow = "CANCELLATION"
                timestamp = parsed_order_book.iloc[i].name
                price = reduced_quote_data.iloc[i+1].values[volume_change_column_index - 1]
                to_append = [timestamp, level, side, price, quantity, flow]
                to_append_series = pd.Series(to_append, index = before_trade_match_order_flow.columns)
                before_trade_match_order_flow  = before_trade_match_order_flow.append(to_append_series, ignore_index = True)
            
    return before_trade_match_order_flow    

In [38]:
before_trade_match_order_flow = classify_changes_parsed_order_book_before_trading_match(reduced_quote_data, parsed_order_book)

In [39]:
before_trade_match_order_flow.head(5)

,Timestamp,Level,Side,Price,Quantity,Flow
0,2016-01-04 09:10:00.043996800,1,ASK,50.95,250.0,LIMIT ORDER
1,2016-01-04 09:10:00.043996800,1,BID,50.92,-72.0,CANCELLATION
2,2016-01-04 09:10:00.043996800,1,ASK,50.95,250.0,CANCELLATION
3,2016-01-04 09:10:00.043996800,3,BID,50.9,-282.0,CANCELLATION
4,2016-01-04 09:10:00.043996800,3,BID,50.9,-100.0,CANCELLATION


In [40]:
before_trade_match_order_flow[before_trade_match_order_flow["Flow"] == "CANCELLATION"].head(20)

,Timestamp,Level,Side,Price,Quantity,Flow
1,2016-01-04 09:10:00.043996800,1,BID,50.92,-72.0,CANCELLATION
2,2016-01-04 09:10:00.043996800,1,ASK,50.95,250.0,CANCELLATION
3,2016-01-04 09:10:00.043996800,3,BID,50.9,-282.0,CANCELLATION
4,2016-01-04 09:10:00.043996800,3,BID,50.9,-100.0,CANCELLATION
6,2016-01-04 09:10:00.043996800,3,BID,50.9,-222.0,CANCELLATION
10,2016-01-04 09:10:00.043996800,4,BID,50.89,-298.0,CANCELLATION
11,2016-01-04 09:10:00.043996800,5,BID,50.88,-188.0,CANCELLATION
12,2016-01-04 09:10:00.043996800,4,ASK,50.98,-100.0,CANCELLATION
13,2016-01-04 09:10:00.043996800,4,ASK,50.98,-52.0,CANCELLATION
14,2016-01-04 09:10:00.043996800,3,BID,50.9,-286.0,CANCELLATION


In [42]:
reduced_quote_data.head(20)

,bid-price-1,bid-volume-1,ask-price-1,ask-volume-1,bid-price-2,bid-volume-2,ask-price-2,ask-volume-2,bid-price-3,bid-volume-3,ask-price-3,ask-volume-3,bid-price-4,bid-volume-4,ask-price-4,ask-volume-4,bid-price-5,bid-volume-5,ask-price-5,ask-volume-5,bid-price-6,bid-volume-6,ask-price-6,ask-volume-6,bid-price-7,bid-volume-7,ask-price-7,ask-volume-7,bid-price-8,bid-volume-8,ask-price-8,ask-volume-8,bid-price-9,bid-volume-9,ask-price-9,ask-volume-9,bid-price-10,bid-volume-10,ask-price-10,ask-volume-10
fulldate,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2016-01-04 09:10:00.043996800,50.92,1344,50.96,767,50.91,322,50.97,327,50.9,1654,50.98,785,50.89,1241,50.99,2313,50.88,1539,51.00,1007,50.87,1699,51.01,1401,50.86,1729,51.02,822,50.85,1203,51.03,1346,50.84,1056,51.04,689,50.83,1223,51.05,1316
2016-01-04 09:10:00.043996800,50.92,1272,50.95,250,50.91,322,50.96,767,50.9,1654,50.97,327,50.89,1241,50.98,785,50.88,1539,50.99,2313,50.87,1699,51,1007,50.86,1729,51.01,1401,50.85,1203,51.02,822,50.84,1056,51.03,1346,50.83,1223,51.04,689
2016-01-04 09:10:00.043996800,50.92,1272,50.96,767,50.91,322,50.97,327,50.9,1654,50.98,785,50.89,1241,50.99,2313,50.88,1539,51.00,1007,50.87,1699,51.01,1401,50.86,1729,51.02,822,50.85,1203,51.03,1346,50.84,1056,51.04,689,50.83,1223,51.05,1316
2016-01-04 09:10:00.043996800,50.92,1272,50.96,767,50.91,322,50.97,327,50.9,1372,50.98,785,50.89,1241,50.99,2313,50.88,1539,51.00,1007,50.87,1699,51.01,1401,50.86,1729,51.02,822,50.85,1203,51.03,1346,50.84,1056,51.04,689,50.83,1223,51.05,1316
2016-01-04 09:10:00.043996800,50.92,1272,50.96,767,50.91,322,50.97,327,50.9,1272,50.98,785,50.89,1241,50.99,2313,50.88,1539,51.00,1007,50.87,1699,51.01,1401,50.86,1729,51.02,822,50.85,1203,51.03,1346,50.84,1056,51.04,689,50.83,1223,51.05,1316
2016-01-04 09:10:00.043996800,50.92,1272,50.95,230,50.91,322,50.96,767,50.9,1272,50.97,327,50.89,1241,50.98,785,50.88,1539,50.99,2313,50.87,1699,51,1007,50.86,1729,51.01,1401,50.85,1203,51.02,822,50.84,1056,51.03,1346,50.83,1223,51.04,689
2016-01-04 09:10:00.043996800,50.92,1272,50.95,230,50.91,322,50.96,767,50.9,1050,50.97,327,50.89,1241,50.98,785,50.88,1539,50.99,2313,50.87,1699,51,1007,50.86,1729,51.01,1401,50.85,1203,51.02,822,50.84,1056,51.03,1346,50.83,1223,51.04,689
2016-01-04 09:10:00.043996800,50.92,1272,50.95,230,50.91,322,50.96,989,50.9,1050,50.97,327,50.89,1241,50.98,785,50.88,1539,50.99,2313,50.87,1699,51,1007,50.86,1729,51.01,1401,50.85,1203,51.02,822,50.84,1056,51.03,1346,50.83,1223,51.04,689
2016-01-04 09:10:00.043996800,50.92,1572,50.95,480,50.91,322,50.96,989,50.9,1050,50.97,327,50.89,943,50.98,785,50.88,1351,50.99,2313,50.87,1699,51,1007,50.86,1729,51.01,1401,50.85,1203,51.02,822,50.84,1056,51.03,1346,50.83,1223,51.04,689
